# Example: Rosenbrock function

For more information of the Rosenbrock function please visit: https://en.wikipedia.org/wiki/Rosenbrock_function

In [1]:
# General imports.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local HMC implementation.
from src.hmc_sampler import HMC

In [2]:
# Define the Rosenbrock function and its gradient.
def rose_func(v, a=1.15, b=0.5):
    x, y = v
    return (a - x)**2 + b*(y - x**2)**2
# _end_def_

def rose_grad(v, a=1.15, b=0.5):
    x, y = v
    return np.array([2.0*(x - a) - 4.0*b*x*(y - x**2),
                     2.0*b*(y - x**2)])
# _end_def_

In [3]:
# Create an HMC object.
rose_hmc = HMC(rose_func, rose_grad, n_omitted=500, n_samples=2_500,
               verbose=True, grad_check=True, rng_seed=911, kappa=50, d_tau=0.02)

# Print out the settings.
print(rose_hmc)

 HMC Id(4815431040): 
 Func(x)=<function rose_func at 0x11f0b84c0> 
 Grad(x)=<function rose_grad at 0x120d89550> 
 Options:
 	n_samples: 2500
	n_omitted: 500
	kappa: 50
	d_tau: 0.02
	grad_check: True
	generalized: False
	verbose: True
	rng_seed: RandomState(MT19937)



In [ ]:
# Run the sampling from x0 starting point.
results = rose_hmc(x0=[0.0, 10.0])

Checking gradients <BEFORE> sampling ... 
Error <BEFORE> = 2.861022947442393e-07
 >>> HMC started 


 Iter=0 - E=3.844 - Acceptance=1.000:  28%|██▊       | 839/3000 [00:01<00:02, 759.50it/s]

In [ ]:
# It is easier to convert the data directly to pandas.
df = pd.DataFrame(results["Samples"])

In [ ]:
# Step size.
delta = 0.01

# Discretized x/y dimensions.
xk = np.arange(-2.0, 4.1, delta)
yk = np.arange(-5.0, 15.5, delta)

# Get the meshgrid.
X, Y = np.meshgrid(xk, yk)

# Evaluate the function.
Z = rose_func([X, Y], a=1.15, b=0.5)

# This is the "real" function.
plt.contourf(X, Y, Z, levels=40);
plt.xlabel("X"); plt.ylabel("Y");
plt.title("Rosenbrock function");

In [ ]:
# Create a large figure.
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(18, 8))

ax[0].contour(X, Y, Z, levels=40);
ax[0].plot(df[0], df[1], 'o--', alpha=0.75);
ax[0].set_xlabel("x"); ax[0].set_ylabel("y");
ax[0].grid(True); ax[0].set_title("Samples");

ax[1].contour(X, Y, Z, levels=40);
ax[1].hexbin(df[0], df[1], bins=100, cmap='gist_heat_r');
ax[1].set_xlabel("x"); ax[1].set_ylabel("y");
ax[1].grid(True); ax[1].set_title("Density");

In [ ]:
# ACF lag value.
lag_n = 40

# Plot the ACF.
plt.stem(rose_hmc.acf(lag_n=lag_n));

plt.grid(True);

plt.title("Autocorrelation");

plt.xlim([-1, lag_n+1])

plt.ylim([-1.5, 1.5])

plt.xticks(range(0, lag_n+1, 5))

plt.xlabel("Lag");